In [1]:
import os

os.chdir('..')

In [2]:
import json
import pandas as pd
import spacy
from spacy import displacy

In [4]:
from NER.EntityExtractor import EntityExtractor

In [5]:
def read_jsonl(data_path):
    with open(data_path, 'r') as f:
        json_list = list(f)
    data = []
    for j in json_list:
        data.append(json.loads(j))
    return pd.DataFrame(data)

def display_entities(text, entities, entity_type=None):
    nlp = spacy.blank("en")
    doc = nlp.make_doc(text)
    ents = []
    for entity in entities:
        if (entity["label"] == entity_type) or (entity_type is None):
            span_start = entity["start"]
            span_end = entity["end"]
            label = entity["label"]
            ent = doc.char_span(span_start, span_end, label=label)
            if ent is None:
                continue
            ents.append(ent)
    doc.ents = ents
    displacy.render(doc, style="ent", jupyter=True)

In [6]:
path = './NER/assets/annotations_test.jsonl'
test = read_jsonl(path)

In [8]:
extractor = EntityExtractor()

In [13]:
for i in range(test.shape[0]):
    print('')
    print(f'Example {i+1}')
    print('')
    row = test.iloc[i]
    display_entities(row.text, row.spans, None)
    print('')
    extractor.get_predictions([row.text])


Example 1




[('Retroviral infection', 'DISEASE')]

Example 2




[('10', 'TARGET')]

Example 3




[('41B', 'TARGET'), ('MKN45', 'TARGET'), ('EGFRMAPKERK12', 'TARGET'), ('PI3KAKT', 'TARGET')]

Example 4




[('Dock1', 'TARGET'), ('Itga4', 'TARGET')]

Example 5



...


[('IL6', 'TARGET'), ('CDX2', 'TARGET'), ('MKN45', 'TARGET'), ('pSTAT3', 'TARGET'), ('NUGC4', 'TARGET'), ('IL6', 'TARGET'), ('CDX2', 'TARGET'), ('SOX2', 'TARGET')]

Example 27




[('hyphal growth', 'DISEASE')]

Example 28




[('antigen1', 'TARGET'), ('Thy1', 'TARGET'), ('glycosylphosphatidylinositol GPIlinked cell surface', 'TARGET'), ('affecting cell', 'DISEASE')]

Example 29




[('PPAR alpha', 'TARGET'), ('Glut2', 'TARGET')]

Example 30




[('type A', 'TARGET'), ('cytokines', 'TARGET')]
